<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# <center>**Analyzing launch site locations with Folium**<center>


The success rate of a launch can depend on many factors, such as the payload mass, the type of orbit, etc. It can also depend on the location and proximity of the launch site—that is, the initial position of the rocket's trajectories. Finding the optimal location to build a launch site undoubtedly involves many factors, and we hope to uncover some of them by analyzing the locations of existing launch sites.


In the previous exploratory data analysis labs, we visualized the SpaceX launch dataset using matplotlib and seaborn and discovered some preliminary correlations between launch site and success rates. In this lab, we will perform more interactive visual analysis using Folium.


## Objectives


This lab contains the following workflow:

- **STEP 1:** We will mark all launch sites on a map
- **STEP 2:** We will mark the successful/failed launches from each site on the map
- **STEP 3:** We will calculate the distances between a launch site and its surroundings

After completing the previous labs, we should be able to identify geographic patterns at the launch sites.

In [1]:
import folium
import wget
import pandas as pd

In [2]:
# Import the Folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import the Folium MousePosition plugin
from folium.plugins import MousePosition
# Import the Folium DivIcon plugin
from folium.features import DivIcon

## STEP 1: We will mark all the launch sites on a map


First, let's try adding the location of each site to a map using the site's latitude and longitude coordinates.


The following dataset named `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.

In [3]:
# Download and read `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at the coordinates of each site.


In [4]:
# Select relevant subcolumns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


The coordinates above are simply numbers that don't give us an intuitive idea of ​​where those launch sites are. If we're good at geography, we can interpret those numbers directly in our minds. Let's visualize those locations by marking them on a map.

First we need to create a `Map` object in the form of a sheet of paper, with an initial central location that will be NASA's Johnson Space Center in Houston, Texas.

In [5]:
# The starting location is NASA's Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circular area with a text label at a specific coordinate. For example,


In [6]:
# We created an orange circle at the coordinates of NASA's Johnson Space Center with a pop-up label showing its name.
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# We created a blue circle at the coordinate of NASA's Johnson Space Center with an icon showing its name.
marker = folium.map.Marker(
    nasa_coordinate,
    # We create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

You should be able to find a small yellow circle near the city of Houston and you can zoom in to see a larger circle.

Now, let's add a circle for each launch site in the `launch_sites` data frame


We will create and add `folium.Circle` and `folium.Marker` for each launch site in the sitemap

In [7]:
# We initialized the map centered on NASA's Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# For each launch site, add a Circle object based on its coordinates (Latitude, Longitude). Also, add the launch site name as a tooltip.
for index, row in launch_sites_df.iterrows():
    circle = folium.Circle([row['Lat'], row['Long']], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['Launch Site']))
    marker = folium.map.Marker(
        [row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

In [8]:
site_map

The generated map with the launch sites marked should look something like the following image:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, we can explore the map by zooming in and out on the marked areas and try to answer the following questions:
- Are all the launch sites close to the equator?
- Are all the launch sites very close to the coast?

# STEP 2: Mark successful/failed launches for each site on the map

Next, let's try to improve the map by adding launch results from each site and see which ones have a high success rate.
Recall that the spacex_df data frame contains detailed launch records, and the `class` column indicates whether this launch was successful or not.

In [9]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, we'll create markers for all launch records. If a launch was successful ``(class=1)``, we'll use a green marker; if it was unsuccessful, we'll use a red marker ``(class=0)``.

Note that a launch only occurs at one of the four launch sites, meaning many launch records will have the exact same coordinate. Marker groups can be a good way to simplify a map that contains many markers with the same coordinate.

First, let's create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()

We will create a new column in the `launch_sites` data frame called `marker_color` to store the marker colors based on the `class` value.


In [11]:
# We create the releases dataframe
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Add the marker_color column
launch_sites['marker_color'] = launch_sites['class'].map({0: 'red', 1: 'green'})

In [12]:
# For each launch point, add a Circle object based on its coordinates (Latitude, Longitude). Also, add the launch point name as a pop-up label.
launch_sites_df = launch_sites.groupby(['Launch Site'], as_index=False).first()
for index, row in launch_sites_df.iterrows():
    circle = folium.Circle([row['Lat'], row['Long']], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(row['Launch Site']))
    marker = folium.map.Marker(
        [row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
            )
        )
    marker_cluster.add_child(circle)
    marker_cluster.add_child(marker)

For each launch result in the `spacex_df` data frame, we add a `folium.Marker` to `marker_cluster`

In [13]:
# Add marker_cluster to the current site map
site_map.add_child(marker_cluster)

# For each row in the spacex_df data frame
# Create a Marker object with its coordinates
# Customize the Marker icon property to indicate whether the launch was successful or unsuccessful.
# For example: icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
 # Create and add a Marker cluster to the site map
    marker = folium.Marker(location=[record['Lat'], record['Long']],)
    marker_cluster.add_child(circle)
    marker_cluster.add_child(marker)

site_map

Our updated map should look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in the marker groups, we should be able to easily identify which launch sites have relatively high success rates.


# STEP 3: We will calculate the distances between a launch site and its surroundings


Next, we need to explore and analyze the vicinity of the launch sites.


First, let's add a `MousePosition` to the map to get the coordinates when you hover over a point. This way, as you explore the map, you can easily find the coordinates of any point of interest (such as a railroad track).


In [14]:
# We added the mouse position to get the coordinates (latitude, longitude) when hovering over the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now we can zoom in on a launch site and explore its proximity to see if we can easily find any railroads, highways, coastlines, etc. We move the mouse to these points and mark their coordinates (shown in the top left) to indicate the distance to the launch site.

We can calculate the distance between two points on the map based on their "Lat" and "Long" values ​​using the following method:


In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

We mark a point on the nearest coast using MousePosition and calculate the distance between the point on the coast and the launch site.

In [16]:
# Coastal coordinates
coastline_lat = 28.56337
coastline_lon = -80.56724
coordinates_coast = (coastline_lat, coastline_lon)
# We Find the coordinates of the launch site
launch_site_lat = 28.56341
launch_site_lon = -80.57677
coordinates_launch = (launch_site_lat, launch_site_lon)
# We calculate the distance between the launch site and the coast
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

After getting the coordinates, we create a `folium.Marker` to display the distance


In [17]:
# We create and add a folium marker at the nearest coastal point you have selected on the map.
# We show the distance between the shore point and the launch point using the icon property.
# For example.
coordinates = [launch_site_lat, launch_site_lon]
distance = distance_coastline
distance_marker = folium.Marker(
   coordinates,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
       )
   )
site_map.add_child(distance_marker)

We draw a `Polyline` between a launch site and the selected point on the coast


In [18]:
# We create a `folium.PolyLine` object using the coastline coordinates and the launch site coordinates
lines = folium.PolyLine(locations= [coordinates_coast, coordinates_launch], weight=1)
site_map.add_child(lines)

Our updated map with the distance line should look like the following screenshot:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


Similarly, we can draw a line between a launch site and its nearest city, railroad, highway, etc. We must first use `MousePosition` to find its coordinates on the map.


A railway map symbol might look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A symbol on a road map might look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A symbol on a city map might look like this:

<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [19]:
# We create a marker with the distance to the nearest city, railroad, highway, etc.
# We draw a line between the marker and the launch point.

from folium.features import DivIcon
from geopy.distance import geodesic

In [20]:
# Launch site coordinates
launch_lat, launch_lon = 28.562302, -80.577356
launch_coords = (launch_lat, launch_lon)

# Coordinates of points of interest
puntos = {
    'Carretera: Samuel C Phillips Pkwy': (28.563197, -80.570891),
    'Ferrocarril: NASA Railroads': (28.572872, -80.585213),
    'Ciudad: Milbourne': (28.513400, -80.681640),
    'Costa más cercana': (28.564900, -80.567800)
}

# We create the map
mapa = folium.Map(location=launch_coords, zoom_start=12)

# Added launch site marker
folium.Marker(
    location=launch_coords,
    popup='Sitio de Lanzamiento',
    icon=folium.Icon(color='green', icon='rocket')
).add_to(mapa)

# We add each point, line and distance marker
for nombre, coords in puntos.items():
    # We calculate distance
    distancia_km = geodesic(launch_coords, coords).km
    
    # We draw the line between the launch site and the point
    folium.PolyLine(
        locations=[launch_coords, coords],
        color='blue',
        weight=2
    ).add_to(mapa)
    
    # We add the marker at the point of interest
    folium.Marker(
        location=coords,
        popup=nombre,
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(mapa)
    
    # Midpoint to place the distance text
    mid_lat = (launch_coords[0] + coords[0]) / 2
    mid_lon = (launch_coords[1] + coords[1]) / 2
    
    # We add text with the distance
    folium.Marker(
        location=[mid_lat, mid_lon],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 12px; color: #d35400;"><b>{:.2f} KM</b></div>'.format(distancia_km),
        )
    ).add_to(mapa)

# We show the map
mapa



### We combined both locations on the map to make it look more dynamic.

In [30]:
def add_site(map, site_name, site_coord, points, icon_color):
    group = folium.FeatureGroup(name=site_name)

    # Launch site marker
    folium.Marker(
        location=site_coord,
        popup=site_name,
        icon=folium.Icon(color=icon_color, icon='rocket')
    ).add_to(group)

    # Add related points
    for point_name, point_coord in points.items():
        # Calculate distance
        distance_km = geodesic(site_coord, point_coord).km

        # Line
        folium.PolyLine(
            locations=[site_coord, point_coord],
            color='blue',
            weight=2
        ).add_to(group)

        # Point marker
        folium.Marker(
            location=point_coord,
            popup=point_name,
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(group)

        # Text with distance
        mid_lat = (site_coord[0] + point_coord[0]) / 2
        mid_lon = (site_coord[1] + point_coord[1]) / 2
        folium.Marker(
            location=[mid_lat, mid_lon],
            icon=DivIcon(
                icon_size=(150, 36),
                icon_anchor=(0, 0),
                html='<div style="font-size: 12px; color: #d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
            )
        ).add_to(group)

    group.add_to(map)

# Create the map centered between the two sites
map = folium.Map(location=[31.5, -100], zoom_start=6)

# --- Site 1: Cape Canaveral ---
cape_coords = (28.562302, -80.577356)
cape_points = {
    'Road: Samuel C Phillips Pkwy': (28.563197, -80.570891),
    'Railroad: NASA Railroads': (28.572872, -80.585213),
    'City: Milbourne': (28.513400, -80.681640),
    'Nearest Coast': (28.564900, -80.567800)
}
add_site(map, "Cape Canaveral", cape_coords, cape_points, icon_color='green')

# --- Site 2: Vandenberg ---
vandenberg_coords = (34.63283, -120.61074)
vandenberg_points = {
    'Road: State Route 246': (34.63800, -120.60250),
    'Railroad: UPRR Railroad': (34.63600, -120.61800),
    'City: Lompoc': (34.6390, -120.4579),
    'Nearest coast': (34.6335, -120.6250)
}
add_site(map, "Vandenberg", vandenberg_coords, vandenberg_points, icon_color='blue')

# Control to toggle sites
folium.LayerControl().add_to(map)

# Show map
map


After drawing the distance lines to the proximities, we can easily answer the following questions:
- Are the launch sites close to railroads?
- Are the launch sites close to highways?
- Are the launch sites close to the coast?
- Are the launch sites kept at a certain distance from cities?

It would appear that the two launch locations share the same specifications in terms of distances to nearby key sites:

* The launch sites are relatively close to the railways. We assume this is due to logistics, operational, and cost-benefit reasons for the company.

* The roads are actually a bit further from the launch sites. We can conclude that if the railways are going to be used, having difficult road access seems reasonable for safety reasons.

* The two launch sites are very close to their respective coasts.

* However, they are very far from cities for logical safety reasons.


# Next steps:

We've now uncovered a lot of interesting information about launch site locations using Folium, in a very interactive way. Next, we'll create a dashboard with Ploty Dash based on detailed launch logs.


<br>

-----------------------------

## Author


<a href="https://www.linkedin.com/in/flavio-aguirre-12784a252/">Flavio Aguirre</a><br>
Data Scientist


Copyright © 2021 IBM Corporation. All rights reserved.
